In [1]:
import pandas as pd
from transformers import pipeline
import google.generativeai as genai
from modules.SummaryManager import EvalSummaryManager

from modules.summarizer import Summarizer
import evaluate

/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
data = pd.read_csv('./sample_data/discharge.csv')

In [ ]:
summarizer = pipeline("summarization", model="Falconsai/medical_summarization")


In [ ]:
text = data.iloc[0]['text']

print(summarizer(text, max_length=2000, min_length=1500, do_sample=False)[0]['summary_text'])

In [ ]:
summarizer = Summarizer(model='gemini')

discharge = '''
    Patient name: John Smith
    DOB: 03/15/1972
    MRN: 12345678
    Admitted: 05/30/2025
    Discharged: 06/04/2025
    Admitting & Discharging Physician: Dr. Emily Thompson

    Patient was admitted with a 3-day history of productive cough, fever up to 39°C, shortness of breath, and pleuritic chest pain. Chest X-ray showed right lower lobe pneumonia. Started on IV ceftriaxone and azithromycin. Given oxygen via nasal cannula to keep saturation above 92%.

    Fever resolved after 3 days. WBC dropped from 14.2 to 8.6. Cultures were negative. Switched to oral amoxicillin-clavulanate on hospital day 4. Blood glucose managed with insulin sliding scale. BP controlled with amlodipine.

    Discharge meds:

        Amoxicillin-clavulanate 875/125 mg PO BID for 3 more days

        Amlodipine 5 mg daily

        Metformin 500 mg BID

        Acetaminophen 500 mg PRN

    Follow-up with primary care in 1 week. Repeat chest X-ray in 6 weeks. Endocrinology referral for diabetes.

    Patient discharged stable, afebrile, eating and walking independently.
'''

summ = summarizer.summ(discharge)

In [ ]:


rouge = evaluate.load('rouge')

results = rouge.compute(predictions=[summ], references=[discharge])

for k, v in results.items():
    print(f'{k}: {v:.4f}')

In [ ]:
bs = evaluate.load('bertscore')
results = bs.compute(predictions=[summ], references=[discharge], lang="en")


In [ ]:
results

In [ ]:
results

In [ ]:
texts = [
    'The patient is a 67-year-old male with a history of congestive heart failure, hypertension, and type 2 diabetes mellitus. He was admitted with worsening shortness of breath and leg swelling. Echocardiogram showed an ejection fraction of 35%. He was started on intravenous diuretics and his symptoms improved. Blood glucose levels were managed with insulin. The patient was counseled on a low-sodium, heart-healthy diet and discharged on Lasix, Metoprolol, and Lisinopril. Follow-up was scheduled with cardiology and endocrinology. He was advised to monitor weight daily and return if symptoms worsen.',
    'Artificial intelligence is reshaping industries worldwide. Companies are investing heavily in AI tools to optimize operations, enhance customer service, and develop new products. From healthcare to finance, AI applications are improving diagnostics, fraud detection, and decision-making. However, concerns remain about job displacement, ethical AI use, and data privacy. Governments are beginning to propose regulatory frameworks to ensure safe development and deployment. Experts emphasize the importance of transparency, accountability, and fairness in AI systems. The global AI market is expected to grow significantly over the next decade, driving innovation while presenting complex challenges for society.',
    'The Industrial Revolution marked a major turning point in history, transforming economies from agrarian to industrial. Beginning in the late 18th century in Britain, it saw the invention of machines such as the spinning jenny and steam engine. These innovations revolutionized textile production and transportation. Urbanization increased as people moved to cities for factory jobs. While industrialization boosted productivity and wealth, it also brought harsh labor conditions and environmental degradation. Social reform movements emerged to address these issues, laying the groundwork for modern labor laws. The revolution eventually spread globally, influencing economic structures and societal organization.',
    'In recent weeks, extreme weather events have impacted multiple countries. Flooding in southern Asia has displaced thousands, while record-breaking heat waves in parts of Europe and North America have strained power grids and healthcare systems. Scientists attribute these patterns to climate change, warning of increasingly frequent and intense events in the future. Governments are responding with emergency aid, infrastructure reviews, and climate adaptation plans. Environmental groups are urging faster action on emissions reductions. These events underscore the urgent need for coordinated international efforts to mitigate climate risks and protect vulnerable populations worldwide.',
    'Photosynthesis is a critical biological process through which green plants, algae, and some bacteria convert light energy into chemical energy. Using sunlight, carbon dioxide from the air, and water from the soil, plants produce glucose and oxygen. This process occurs primarily in the chloroplasts of plant cells and involves two main stages: the light-dependent reactions and the Calvin cycle. Photosynthesis not only sustains plant growth but also forms the foundation of most food chains. Additionally, it plays a vital role in regulating atmospheric carbon dioxide levels, contributing to climate stability and supporting life on Earth.',
]

In [ ]:
discharges = data[:25]['text'].tolist()

In [ ]:
eval = EvalSummaryManager()
eval.add_model('bart')
eval.add_model('t5')
eval.add_model('gemini')
results = eval.everything_eval(discharges)
results

In [ ]:
results.to_csv('output_metricas_mimic.csv', index=False)

In [ ]:
results

In [ ]:
import google.generativeai as genai
import json

with open('config/cred.json') as f:
    config = json.load(f)
    api_key = config['api_key']

genai.configure(api_key=api_key)

model = genai.GenerativeModel(model_name='gemini-2.5-flash')

response = model.generate_content('Explain how AI works in a few words')
print(response.text)


In [ ]:
prompt = '''

summarize the following text:
The patient is a 67-year-old male with a history of congestive heart failure, hypertension, and type 2 diabetes mellitus. He was admitted with worsening shortness of breath and leg swelling. Echocardiogram showed an ejection fraction of 35%. He was started on intravenous diuretics and his symptoms improved. Blood glucose levels were managed with insulin. The patient was counseled on a low-sodium, heart-healthy diet and discharged on Lasix, Metoprolol, and Lisinopril. Follow-up was scheduled with cardiology and endocrinology. He was advised to monitor weight daily and return if symptoms worsen.
'''


model.generate_content(prompt).text

In [ ]:
from google.cloud import bigquery

# Path to your service account key file
key_path = "mimic-test-key.json"

# Create a client using the key file
client = bigquery.Client.from_service_account_json(key_path)

# Your SQL query
query = """
SELECT *
FROM `physionet-data.mimiciv_note.discharge`
LIMIT 10
"""

# Run the query
query_job = client.query(query)

# Convert to DataFrame
df = query_job.to_dataframe()

print(df.head())


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='mimic-test-465504')

df = client.query("""
SELECT *
FROM `physionet-data.mimiciv_note.discharge`
LIMIT 10
""").to_dataframe()

print(df)


In [5]:
discharge = '''
 
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
Percocet
 
Attending: ___.
 
Chief Complaint:
abdominal fullness and discomfort
 
Major Surgical or Invasive Procedure:
___ diagnostic paracentesis
___ therapeutic paracentesis

 
History of Present Illness:
___ with HIV on HAART, COPD, HCV cirrhosis complicated by 
ascites and HE admitted with abdominal distention and pain. She 
was admitted to ___ for the same symptoms 
recently and had 3L fluid removed (no SBP) three days ago and 
felt better. Since discharge, her abdomen has become 
increasingly distended with pain. This feels similar to prior 
episodes of ascites.  
Her diuretics were recently decreased on ___ due to worsening 
hyponatremia 128 and hyperkalemia 5.1. Patient states she has 
been compliant with her HIV and diuretic medications but never 
filled out the lactulose prescription. She states she has had 
___ BMs daily at home. She has had some visual hallucinations 
and forgetfulness. Her appetite has been poor.  
In the ED, initial vitals were 98.9 88 116/88 18 97% RA. CBC 
near baseline, INR 1.4, Na 125, Cr 0.6. AST and ALT mildly above 
baseline 182 and 126 and albumin 2.8. Diagnostic para with 225 
WBC, 7% PMN, total protein 0.3. UA with few bact, 6 WBC, mod 
leuk, neg nitr, but contaminated with 6 epi. CXR clear. RUQ US 
with no PV thrombus, moderate ascites. She was given ondansetron 
4mg IV and morphine 2.5mg IV x1 in the ED.  
On the floor, she is feeling improved but still has abdominal 
distention and discomfort.  
ROS: +Abdominal distention and pain. No black/bloody stools. No 
___ pain or swelling. No fevers or chills. Denies chest pain, 
nausea, vomiting. No dysuria or frequency. 
 
Past Medical History:
1. HCV Cirrhosis  
2. No history of abnormal Pap smears.  
3. She had calcification in her breast, which was removed  
previously and per patient not, it was benign.  
4. For HIV disease, she is being followed by Dr. ___ Dr.  
___.  
5. COPD  
6. Past history of smoking.  
7. She also had a skin lesion, which was biopsied and showed  
skin cancer per patient report and is scheduled for a complete  
removal of the skin lesion in ___ of this year.  
8. She also had another lesion in her forehead with purple  
discoloration. It was biopsied to exclude the possibility of  
___'s sarcoma, the results is pending.  
9. A 15 mm hypoechoic lesion on her ultrasound on ___  
and is being monitored by an MRI.  
10. History of dysplasia of anus in ___.  
11. Bipolar affective disorder, currently manic, mild, and PTSD. 
 
12. History of cocaine and heroin use.  

 
Social History:
___
Family History:
She a total of five siblings, but she is not  talking to most of 
them. She only has one brother that she is in  touch with and 
lives in ___. She is not aware of any known GI or liver 
disease in her family.  
 
Physical Exam:
ADMISSION PHYSICAL EXAM:  
VS: T98.1 105/57 79 20 97RA 44.6kg  
GENERAL: Thin chronically ill appearing woman in no acute 
distress  
HEENT: Sclera anicteric, MMM, no oral lesions  
HEART: RRR, normal S1 S2, no murmurs  
LUNGS: Clear, no wheezes, rales, or rhonchi  
ABD: Significant distention with visible veins, bulging flanks, 
nontender to palpation, tympanitic on percussion, normal bowel 
sounds  
EXT: no ___ edema, 2+ DP and ___ pulses  
NEURO: alert and oriented, not confused, no asterixis

DISCHARGE PE:
VS: T 98.4 BP 95/55 (SBP ___ HR 80 RR 18 O2 95RA  
I/O 240/150 this am  
GENERAL: Thin chronically ill appearing woman in no acute 
distress  
HEENT: Sclera anicteric, MMM, no oral lesions  
HEART: RRR, normal S1 S2, no murmurs  
LUNGS: Clear, no wheezes, rales, or rhonchi  
ABD: Significant distention with visible veins, bulging flanks, 
nontender to palpation, tympanitic on percussion, normal bowel 
sounds  
EXT: no ___ edema, 2+ DP and ___ pulses  
NEURO: alert and oriented, not confused, no asterixis
 
Pertinent Results:
LABS ON ADMISSION:
___ 04:10PM BLOOD ___ 
___ Plt ___
___ 04:10PM BLOOD ___ 
___
___ 04:10PM BLOOD ___ 
___
___ 04:10PM BLOOD ___ 
___
___ 04:10PM BLOOD ___
___ 04:39PM BLOOD ___

LABS ON DISCHARGE:
___ 05:10AM BLOOD ___ 
___ Plt ___
___ 05:10AM BLOOD ___ ___
___ 05:10AM BLOOD ___ 
___
___ 05:10AM BLOOD ___ 
___
___ 05:10AM BLOOD ___

MICRO:
___ 10:39 pm URINE      Source: ___. 

                            **FINAL REPORT ___

   URINE CULTURE (Final ___: 
      MIXED BACTERIAL FLORA ( >= 3 COLONY TYPES), CONSISTENT 
WITH SKIN
      AND/OR GENITAL CONTAMINATION. 

___ 7:00 pm PERITONEAL FLUID      PERITONEAL FLUID. 

   GRAM STAIN (Final ___: 
      1+    (<1 per 1000X FIELD):   POLYMORPHONUCLEAR 
LEUKOCYTES. 
      NO MICROORGANISMS SEEN. 
      This is a concentrated smear made by cytospin method, 
please refer to
      hematology for a quantitative white blood cell count.. 

   FLUID CULTURE (Final ___:    NO GROWTH. 

   ANAEROBIC CULTURE (Preliminary):    NO GROWTH.

___ 7:00 pm PERITONEAL FLUID      PERITONEAL FLUID. 

   GRAM STAIN (Final ___: 
      1+    (<1 per 1000X FIELD):   POLYMORPHONUCLEAR 
LEUKOCYTES. 
      NO MICROORGANISMS SEEN. 
      This is a concentrated smear made by cytospin method, 
please refer to
      hematology for a quantitative white blood cell count.. 

   FLUID CULTURE (Final ___:    NO GROWTH. 

   ANAEROBIC CULTURE (Preliminary):    NO GROWTH. 

Diagnositc Para:
___ 07:00PM ASCITES ___ 
___
___ 07:00PM ASCITES ___

IMAGING:
___ CXR- No acute cardiopulmonary abnormality.  
___ RUQ US-  
1. Extremely coarse and nodular liver echotexture consistent 
with a history of cirrhosis.  
2. Moderate ascites.  
3. Patent portal vein.
 
Brief Hospital Course:
___ with HIV on HAART, HCV cirrhosis with ascites and HE, h/o 
IVDU, COPD, bipolar disorder presents with abdominal discomfort 
due to ___ ascites.  
 
# ASCITES. Now diuretic refractory given last tap was three days 
ago with 3L removed and she has already built up moderate 
ascites. Infectious workup negative, with CXR clear, UA 
contaminated but not grossly positive so will f/u culture, 
diagnostic para with only 225 WBC, RUQ US with no PV thrombus. 
Compliant with diuretics but not following low sodium diet or 
fluid restriction. Dr. ___ discussed possible TIPS in 
the office but due to lung disease, that was on hold pending 
further cardiac evaluation. Diuretics were recently decreased 
due to hyponatremia and hyperkalemia. Held spironolactone for 
now due to K 5.2 and increased lasix 20 -> 40. No evidence of 
severe hyponatremia (Na<120) or renal failure Cr>2.0 to stop 
diuretics at present. Diagnostic paracentesis negative for 
infection. Ascitic total protein 0.3 so warrants SBP prophylaxis 
(<1.0) and fortunately already on Bactrim for PCP prophylaxis 
which would be appropriate for SBP ppx also. Patient did admit 
to eating pizza and some ___ food prior to 
admission. She had therapeutic paracentesis with 4.3L removed 
and received 37.5G albumin IV post procedure. She felt much 
better with resolution of abdominal discomfort. Patient is 
scheduled for repeat paracentesis as outpatient on ___.  

# HEPATIC ENCEPHALOPATHY. History of HE from Hep C cirrhosis. 
Now with mild encephalopathy (hallucinations and forgetfulness) 
due to medication noncompliance, but not acutely encephalopathic 
and without asterixis on exam. Infectious workup negative thus 
far. Continue lactulose 30mL TID and titrate to 3 BMs daily and 
continue rifaximin 550mg BID. 
 
# HYPONATREMIA. Na 125 on admission, 128 four days ago, and 135 
one month ago. Likely due to third spacing from worsening 
ascites and fluid overload. 1.5L fluid restriction, low salt 
diet. S/p therapeutic paracentesis with albumin replacement.

# CIRRHOSIS, HEPATITIS C. MELD score of 10 and Child's ___ 
class B on this admission. Now decompensated due to ascites. 
Hepatitis C genotype IIIB. Dr. ___ starting 
___ and ___ with patient in clinic and the 
insurance process was started by her office. No history of EGD, 
needs this as outpatient for varices screening.  
 
# NUTRITION. Unclear if truly compliant with low salt diet. Poor 
oral intake. Low albumin 2.8 on admission. Met with nutrition. 
 
# COAGULOPATHY. INR 1.4 four days ago. No evidence of active 
bleeding. Very mild thrombocytopenia with plts 143.  
 
# HIV. Most recent CD4 173. On HAART. No established ID 
provider. Continue Truvada and Isentress, Bactrim DS daily for 
PCP ___. Needs outpatient ID appointment  

# COPD. Stable. States she is on intermittent home O2 for 
comfort at night and with abdominal distentiom. Continued home 
COPD meds and home O2 as needed 

**Transitional Issues**
- Discontinued spironolactone ___ elevated potassium
- Increased furosemide to 40mg daily
- Please recheck electrolytes at next visit
- Had paracentesis ___ with 4.3 L removed, received 37.5G 
albumin
- Needs outpatient ID provider
- ___ needs more frequent paracentesis
 
Medications on Admission:
The Preadmission Medication list is accurate and complete.
1. Albuterol Inhaler 2 PUFF IH Q6H:PRN wheezing, SOB 
2. ___ (Truvada) 1 TAB PO DAILY 
3. Furosemide 20 mg PO DAILY 
4. Raltegravir 400 mg PO BID 
5. Spironolactone 50 mg PO DAILY 
6. Acetaminophen 500 mg PO Q6H:PRN pain,fever 
7. Tiotropium Bromide 1 CAP IH DAILY 
8. Rifaximin 550 mg PO BID 
9. Calcium Carbonate 1250 mg PO BID 
10. Lactulose 15 mL PO TID 
11. Sulfameth/Trimethoprim DS 1 TAB PO DAILY 

 
Discharge Medications:
1. Acetaminophen 500 mg PO Q6H:PRN pain,fever 
2. Albuterol Inhaler 2 PUFF IH Q6H:PRN wheezing, SOB 
3. Calcium Carbonate 1250 mg PO BID 
4. ___ (Truvada) 1 TAB PO DAILY 
5. Furosemide 40 mg PO DAILY 
6. Lactulose 15 mL PO TID 
7. Raltegravir 400 mg PO BID 
8. Rifaximin 550 mg PO BID 
9. Sulfameth/Trimethoprim DS 1 TAB PO DAILY 
10. Tiotropium Bromide 1 CAP IH DAILY 

 
Discharge Disposition:
Home
 
Discharge Diagnosis:
Primary:  diuretic refractory ascites
Secondary: HCV cirrhosis, HIV, hyponatremia, COPD

 
Discharge Condition:
Mental Status: Clear and coherent.
Level of Consciousness: Alert and interactive.
Activity Status: Ambulatory - Independent.

 
Discharge Instructions:
Dear ___,

___ was a pleasure to take care of you at ___ 
___. You were admitted with abdominal fullness and 
pain from your ascites. You had a diagnostic and therapeutic 
paracentesis with 4.3 L removed. Your spironolactone was 
discontinued because your potassium was high. Your lasix was 
increased to 40mg daily. You are scheduled for another 
paracentesis on ___ prior to your other appointments that day. 
Please call tomorrow to find out the time of the paracentesis. 
Please continue to follow a low sodium diet and fluid 
restriction. You should call your liver doctor or return to the 
emergency room if you have abdominal pain, fever, chills, 
confusion, or other concerning symptoms.

Sincerely,
Your ___ medical team
 
Followup Instructions:
___


'''

In [6]:
prompt = '''

    Summarize the following discharge note. Don't include visual elements in the text

''' + '\n' + discharge

In [ ]:
genai.configure(api_key="")

model = genai.GenerativeModel("gemini-2.0-flash")

summary_response = model.generate_content(prompt)

print(summary_response.text)


The patient, a female with a history of HIV on HAART, HCV cirrhosis complicated by ascites and hepatic encephalopathy (HE), COPD, bipolar disorder, and past IV drug use, was admitted for abdominal distention and pain due to ascites. She had a recent paracentesis three days prior to admission. Her diuretics were decreased due to hyponatremia and hyperkalemia. On admission, she reported abdominal distention and pain, visual hallucinations, forgetfulness, and poor appetite. Initial labs showed hyponatremia, mildly elevated AST/ALT, and a low albumin. Diagnostic paracentesis showed a low WBC count. RUQ ultrasound revealed no portal vein thrombus and moderate ascites. She improved during her hospital stay.

Pertinent findings included a negative infectious workup, a UA contaminated with skin flora, and a diagnostic paracentesis that was negative for infection. She had a therapeutic paracentesis with 4.3L of fluid removed and received albumin. She is scheduled for a repeat paracentesis as an

In [ ]:
decomposition_prompt = '''


You are a medical information analyst. Your task is to parse the entire medical summary below and decompose it into a complete list of fine-grained, verifiable claims. These claims will be used to fact-check the summary against a full patient record.

Each claim must be a single, self-contained medical fact, finding, instruction, or patient-reported statement.

**Extraction Rules:**
1.  **Split Lists:** Decompose lists of conditions, symptoms, findings, or plans into separate claims (e.g., "history of A, B, and C" becomes three separate claims).
2.  **Keep Instructions Intact:** A single medication (e.g., "Furosemide 40mg daily") is one complete claim. Do not split the drug from its dosage.
3.  **Be Self-Contained:** Resolve all pronouns (e.g., "She," "He") to "The patient."
4.  **Extract All Details:** Capture demographics, past medical history (PMH), symptoms, lab results, imaging findings, diagnoses, medications, and follow-up plans.
5.  **Output Format:** Provide the output as a simple list of claims one in each line.

---
### Example 1

**Full Summary:**
The patient, a female with a history of HCV cirrhosis with ascites, HIV on ART, IV drug use, COPD, bipolar disorder, and PTSD, presented with worsening abdominal distension and pain. She had discontinued her Lasix and Spironolactone prior to admission due to feeling that they weren't effective and wanting to avoid "chemicals."During hospitalization, the patient's ascites was managed with diuretics (Furosemide 40mg and Spironolactone 50mg daily). Paracentesis was attempted in the ED but was unsuccessful. Lab results showed elevated liver enzymes (ALT, AST, Alk Phos, Total Bili), and a low platelet count. A CXR showed no acute cardiopulmonary process. Ultrasound revealed liver nodularity, signs of portal hypertension (ascites and splenomegaly), and cholelithiasis. The patient was scheduled for follow-up with her current PCP, a new PCP (Dr. ___), and the Liver Clinic for EGD and ___. She was discharged home in a clear and coherent mental state, alert, interactive, and ambulatory, with a diagnosis of ascites from portal hypertension. She was instructed to continue taking Furosemide 40mg and Spironolactone 50mg daily, adhere to a low-salt diet, and follow up with the Liver Clinic.


**Claims:**
The patient is female.

The patient has a history of HCV cirrhosis with ascites.

The patient has a history of HIV on ART.

The patient has a history of IV drug use.

The patient has a history of COPD.

The patient has a history of bipolar disorder.

The patient has a history of PTSD.

The patient presented with worsening abdominal distension.

The patient presented with worsening abdominal pain.

The patient had discontinued her Lasix prior to admission.

The patient had discontinued her Spironolactone prior to admission.

The patient's reason for discontinuing was feeling [the drugs] weren't effective.

The patient's reason for discontinuing was wanting to avoid "chemicals."

The patient's ascites was managed with Furosemide 40mg daily during hospitalization.

The patient's ascites was managed with Spironolactone 50mg daily during hospitalization.

Paracentesis was attempted in the ED.

The paracentesis attempt in the ED was unsuccessful.

Lab results showed elevated liver enzymes (ALT, AST, Alk Phos, Total Bili).

Lab results showed a low platelet count.

A CXR showed no acute cardiopulmonary process.

Ultrasound revealed liver nodularity.

Ultrasound revealed signs of portal hypertension (ascites and splenomegaly).

Ultrasound revealed cholelithiasis.

The patient was scheduled for follow-up with her current PCP.

The patient was scheduled for follow-up with a new PCP (Dr. ___).

The patient was scheduled for follow-up with the Liver Clinic for EGD.

The patient was scheduled for follow-up with the Liver Clinic for ___.

The patient was discharged home.

The patient's mental state at discharge was clear and coherent.

The patient's status at discharge was alert.

The patient's status at discharge was interactive.

The patient's status at discharge was ambulatory.

The patient's diagnosis was ascites from portal hypertension.

The patient was instructed to continue taking Furosemide 40mg daily.

The patient was instructed to continue taking Spironolactone 50mg daily.

The patient was instructed to adhere to a low-salt diet.

The patient was instructed to follow up with the Liver Clinic.

---
### New summary (YOUR TASK)

**Full Summary:**
''' + '\n' + summary_response.text


In [14]:
decomp_response = model.generate_content(decomposition_prompt)

In [16]:
print(decomp_response.text)

The patient is female.
The patient has a history of HIV on HAART.
The patient has a history of HCV cirrhosis complicated by ascites.
The patient has a history of HCV cirrhosis complicated by hepatic encephalopathy (HE).
The patient has a history of COPD.
The patient has a history of bipolar disorder.
The patient has a history of past IV drug use.
The patient was admitted for abdominal distention.
The patient was admitted for abdominal pain due to ascites.
The patient had a recent paracentesis three days prior to admission.
The patient's diuretics were decreased due to hyponatremia.
The patient's diuretics were decreased due to hyperkalemia.
On admission, she reported abdominal distention.
On admission, she reported abdominal pain.
On admission, she reported visual hallucinations.
On admission, she reported forgetfulness.
On admission, she reported poor appetite.
Initial labs showed hyponatremia.
Initial labs showed mildly elevated AST/ALT.
Initial labs showed a low albumin.
Diagnostic 

In [1]:
from transformers import pipeline

# nli = pipeline("text-classification", model="emilyalsentzer/Bio_ClinicalBERT")
nli = pipeline("text-classification", model="facebook/bart-large-mnli")

premise = 'The patient, a female with a history of HIV on HAART, HCV cirrhosis complicated by ascites and hepatic encephalopathy (HE), COPD, bipolar disorder, and past IV drug use, was admitted for abdominal distention and pain due to ascites.'
claim = 'The patient is a guy'

result = nli(f"{premise} </s> {claim}")
print(result)


/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads 

[{'label': 'contradiction', 'score': 0.9996641874313354}]
